In [5]:
import numpy as np
import nolds
import struct
import os

def compute_lyapunov_exponents(filename="data.bin"):
    """
    Reads 16-bit data from a binary file, converts it to numbers,
    and computes Lyapunov exponents using nolds.
    """
    # Use numpy.fromfile for efficient reading of binary data
    # 'int16' specifies 16-bit signed integers
    try:
        # Open in binary read mode ('rb')
        with open(filename, 'rb') as f:
            data_array = np.fromfile(f, dtype=np.int16)
        
        # Convert to float array, as nolds functions expect float data
        data_float = data_array[:1000].astype(float)

        print(f"Read {len(data_float)} data points from {filename}")
        
        # --- Compute Largest Lyapunov Exponent using Rosenstein algorithm ---
        # The function `lyap_r` typically returns a single float value
        lle_rosenstein = nolds.lyap_r(data_float, emb_dim=10, lag=1, tau=1)
        print(f"\nLargest Lyapunov Exponent (Rosenstein): {lle_rosenstein:.4f}")

        # --- Compute Lyapunov Spectrum using Eckmann algorithm ---
        # The function `lyap_e` returns a list/array of all exponents
        # The first one is typically the largest
        lle_eckmann_spectrum = nolds.lyap_e(data_float, emb_dim=10, tau=1)
        print(f"Lyapunov Spectrum (Eckmann): {lle_eckmann_spectrum}")
        print(f"Largest Lyapunov Exponent (Eckmann): {lle_eckmann_spectrum[0]:.4f}")

    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# --- Main Execution ---
if __name__ == "__main__":
    data_file_name = "final_1M_bits.txt"
    
    # 2. Process the file and calculate the exponents
    compute_lyapunov_exponents(data_file_name)
    
    # Optional: clean up the dummy file
    # os.remove(data_file_name)
    # print(f"\nRemoved dummy file: {data_file_name}")



Read 1000 data points from final_1M_bits.txt


/home/nirjhar/Python Codes/Fourier/.venv/lib/python3.12/site-packages/nolds/measures.py:263: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 250
  warnings.warn(msg.format(min_tsep), RuntimeWarning)
/home/nirjhar/Python Codes/Fourier/.venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)



Largest Lyapunov Exponent (Rosenstein): 0.0004
Lyapunov Spectrum (Eckmann): [ 0.32964901  0.07372536 -0.13173171 -0.42766166]
Largest Lyapunov Exponent (Eckmann): 0.3296


In [6]:
import math
from collections import Counter

# =========================
# Read bitstream
# =========================
def read_bitstream(filename):
    with open(filename, "r") as f:
        data = f.read()
    bits = [b for b in data if b in ('0', '1')]
    if len(bits) % 16 != 0:
        raise ValueError("Bitstream length is not a multiple of 16.")
    return bits

# =========================
# Convert bits → 16-bit samples
# =========================
def bits_to_samples(bits):
    samples = []
    for i in range(0, len(bits), 16):
        sample = ''.join(bits[i:i+16])
        samples.append(sample)
    return samples

# =========================
# Min-entropy estimator
# =========================
def min_entropy(symbols):
    N = len(symbols)
    counts = Counter(symbols)
    p_max = max(c / N for c in counts.values())
    return -math.log2(p_max)

# =========================
# Block min-entropy (sample-level)
# =========================
def block_min_entropy(samples, block_size):
    if len(samples) < block_size:
        raise ValueError("Not enough samples for chosen block size.")
    blocks = [
        tuple(samples[i:i+block_size])
        for i in range(0, len(samples) - block_size + 1, block_size)
    ]
    H_block = min_entropy(blocks)
    return H_block / block_size

# =========================
# Main
# =========================
if __name__ == "__main__":
    filename = "final_1M_bits.txt"     # your file
    block_size = 4            # samples per block (try 2, 4, 8)

    bits = read_bitstream(filename)
    samples = bits_to_samples(bits)

    H_sample = min_entropy(samples)
    H_bit = H_sample / 16

    H_block_sample = block_min_entropy(samples, block_size)
    H_block_bit = H_block_sample / 16

    print("========== Min-Entropy Estimation ==========")
    print(f"Total bits analysed           : {len(bits)}")
    print(f"Total sensor samples          : {len(samples)}")
    print(f"Min-entropy per 16-bit sample : {H_sample:.6f} bits")
    print(f"Min-entropy per bit           : {H_bit:.6f} bits/bit")
    print(f"Block size                    : {block_size} samples")
    print(f"Block min-entropy per sample  : {H_block_sample:.6f} bits")
    print(f"Block min-entropy per bit     : {H_block_bit:.6f} bits/bit")
    print("============================================")


========== Min-Entropy Estimation ==========
Total bits analysed           : 10000000
Total sensor samples          : 625000
Min-entropy per 16-bit sample : 14.609640 bits
Min-entropy per bit           : 0.913103 bits/bit
Block size                    : 4 samples
Block min-entropy per sample  : 4.313374 bits
Block min-entropy per bit     : 0.269586 bits/bit
